In [203]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd


In [204]:
cardClass = '_1xHGtK'
nameCard='_2WkVRV'
descriptionClass='IRpwTa'
priceClass="_30jeq"
imageClass='CXW8mj'

itemName ='%20'.join(input().split())


In [205]:
itemName= 'pants'
itemName='%20'.join(itemName.split())


In [206]:
itemPage=requests.get(f'https://www.flipkart.com/search?q={itemName}').content
idk = BeautifulSoup(itemPage,'html.parser')
print(idk.prettify())


<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://rukminim1.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.905c37.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.8f4f44.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="102988293558" property="fb:page_id"/>
  <meta content="658873552,624500995,100000233612389" property="fb:admins"/>
  <meta content="noodp" name="robots"/>
  <link href="https:///www/promos/new/20150528-140547-favicon-retina.ico" rel="shortcut icon"/>
  <link href="/osdd.xml?v=2" rel="search" type="application/opensearchdescription+xml"/>
  <meta content="website" property="og:type"/>
  <meta content="Flipkart.com" name="og_site_name" property="og:site_name"/>
  <link href="/apple-touch-icon-57x

In [207]:
def setlist(item,nameCard):
    name=item.find('div',class_=nameCard).text
    dis=item.find('a',class_=descriptionClass)['title']
    return [name,dis ]
    

In [208]:
file=open('temp.txt','w')

In [209]:
itemCards=idk.find_all('div',class_=cardClass)


In [210]:
itemList = [setlist(item,nameCard) for item in itemCards]

print(itemList)
a=pd.DataFrame(itemList)
a.to_excel('data.xlsx')


[['PETER ENGLAND', 'Men Skinny Fit Green Cotton Blend Trousers'], ['PETER ENGLAND', 'Men Slim Fit Black Cotton Blend Trousers'], ['INDIAN TERRAIN', 'Brooklyn Men Slim Fit Black Cotton Blend Trousers'], ['PARK AVENUE', 'Men Regular Fit Black Viscose Rayon Trousers'], ['Roadster', 'Men Slim Fit Beige Cotton Blend Trousers'], ['Allen Solly', 'Men Slim Fit Grey Cotton Blend Trousers'], ['ELANHOOD', 'Men Regular Fit Black Cotton Blend Trousers'], ['PETER ENGLAND', 'Men Regular Fit Dark Green Polycotton Trousers'], ['ELANHOOD', 'Men Regular Fit Black Cotton Blend Trousers'], ['KOTTY', 'Women Regular Fit Dark Green Viscose Rayon Trousers'], ['ADDYVERO', 'Women Regular Fit Black Cotton Blend Trousers'], ['U.S. POLO ASSN.', 'Men Slim Fit Grey Pure Cotton Trousers'], ['KOTTY', 'Women Regular Fit Black Viscose Rayon Trousers'], ['KOTTY', 'Women Regular Fit Orange Viscose Rayon Trousers'], ['KOTTY', 'Women Regular Fit Black Cotton Blend Trousers'], ['PETER ENGLAND', 'Men Skinny Fit Dark Blue Cotto